## Import libraries

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import time
import warnings
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn import datasets, linear_model
from numpy import newaxis
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers import BatchNormalization
from keras.models import Sequential
from keras import regularizers
import keras.backend as K
import statsmodels.formula.api as smf
from matplotlib import pyplot as plt
from keras.callbacks import ModelCheckpoint

plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 10)

# prevent tensorflow from allocating the entire GPU memory at once
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #Hide messy TensorFlow warnings
#warnings.filterwarnings("ignore") #Hide messy Numpy warnings

%autosave 60

Using TensorFlow backend.


Autosaving every 60 seconds


## Load data

In [2]:
def load_data(filename, num_lags, bootstrap_size):
    assert bootstrap_size > num_lags
    
    # read data from file
    f = open(filename)
    series = []
    removed_seasonality = [] 
    removed_std = []
    missings = []
    for line in f:
        splt = line.split(",")
        series.append(float(splt[0]))
        removed_seasonality.append(float(splt[1]))
        removed_std.append(float(splt[2]))
        missings.append(int(splt[3]))
    series = np.array(series)
    removed_seasonality = np.array(removed_seasonality)
    removed_std = np.array(removed_std)
    missings = np.array(missings)
    f.close()

    # generate lags
    X = []
    for i in range(bootstrap_size, len(series)):
        X.append(series[i-num_lags:i])
    X = np.array(X)
    
    y = series[bootstrap_size:]
    removed_seasonality = removed_seasonality[bootstrap_size:]
    removed_std = removed_std[bootstrap_size:]
    missings = missings[bootstrap_size:]
    assert X.shape[0] == y.shape[0]

    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    return X, y, removed_seasonality, removed_std, missings

In [3]:
PLACE_IDS_ORDERED_FROM_NORTH_TO_SOUTH = [
    'ChIJZaR1M1hSUkYRxP0WkwYYy_k',
    'ChIJj1RhMlhSUkYRxwx00g4P0QE',
    'ChIJuYkcKlhSUkYRFPCipW5rTvU',
    'ChIJP6TdhFdSUkYRdrsWKXZMAs8',
    'ChIJf9Y9sFdSUkYRmaDEJhCweGc',
    'ChIJozaGTFZSUkYRNtWl2AGUPkI',
    'ChIJ4QuVTlZSUkYRRDRPcHdYULQ',
    'ChIJBTt5RlZSUkYR_SyA8BgiwaM',
    'ChIJbcDEbFZSUkYRcnQFsIj5j5U'
]

In [4]:
# read data from different places
NUM_LAGS = 50
bootstrap_size = 12*24*1
ids = []
removed_seasonality_all = []
removed_std_all = []
missings_all = []
X_all = []
y_all = []
for fname in map(lambda place_id: place_id + '.csv', PLACE_IDS_ORDERED_FROM_NORTH_TO_SOUTH):
    print "reading data for place id:", fname[-6:-4]
    X, y, removed_seasonality, removed_std, missings = load_data('norrecampus/'+fname, NUM_LAGS, bootstrap_size)
    ids.append(fname[-6:-4])
    X_all.append(X)
    y_all.append(y)
    removed_seasonality_all.append(removed_seasonality)
    removed_std_all.append(removed_std)
    missings_all.append(missings)
X_all = np.array(X_all)
y_all = np.array(y_all)
print X_all.shape
print y_all.shape
n_places = len(ids)
n_instances = X_all.shape[1]
print "n_instances:", n_instances
print "n_places:", n_places
print "n_lags:", NUM_LAGS

reading data for place id: _k
reading data for place id: QE
reading data for place id: vU
reading data for place id: s8
reading data for place id: Gc
reading data for place id: kI
reading data for place id: LQ
reading data for place id: aM
reading data for place id: 5U
(9, 51552, 50, 1)
(9, 51552)
n_instances: 51552
n_places: 9
n_lags: 50


In [5]:
# reshape data
X = np.swapaxes(X_all, 0, 1)
X = np.swapaxes(X, 1, 2)
X = X[:,:,:,:,np.newaxis]
print X.shape
y = np.swapaxes(y_all, 0, 1)
y = y[:,np.newaxis,:,np.newaxis,np.newaxis]
print y.shape

STEPS_AHEAD = 1
X = X[:,:NUM_LAGS-STEPS_AHEAD+1,:,:]
print X.shape

(51552, 50, 9, 1, 1)
(51552, 1, 9, 1, 1)
(51552, 50, 9, 1, 1)


## Train/test split

In [6]:
n_train = 12*24*89
n_test = n_train + 12*24*90
X_train = X[:n_train,:]
y_train = y[:n_train]
X_test = X[n_train:n_test,:]
y_test = y[n_train:n_test]
print X_train.shape
print y_train.shape
print X_test.shape
print y_test.shape

(25632, 50, 9, 1, 1)
(25632, 1, 9, 1, 1)
(25920, 50, 9, 1, 1)
(25920, 1, 9, 1, 1)


## Evaluation functions

In [7]:
def compute_error(trues, predicted):
    corr = np.corrcoef(predicted, trues)[0,1]
    mae = np.mean(np.abs(predicted - trues))
    mse = np.mean((predicted - trues)**2)
    rae = np.sum(np.abs(predicted - trues)) / np.sum(np.abs(trues - np.mean(trues)))
    rmse = np.sqrt(np.mean((predicted - trues)**2))
    r2 = max(0, 1 - np.sum((trues-predicted)**2) / np.sum((trues - np.mean(trues))**2))
    return corr, mae, mse, rae, rmse, r2

In [8]:
def compute_error_filtered(trues, predicted, filt):
    trues = trues[filt]
    predicted = predicted[filt]
    corr = np.corrcoef(predicted, trues)[0,1]
    mae = np.mean(np.abs(predicted - trues))
    mse = np.mean((predicted - trues)**2)
    rae = np.sum(np.abs(predicted - trues)) / np.sum(np.abs(trues - np.mean(trues)))
    rmse = np.sqrt(np.mean((predicted - trues)**2))
    r2 = max(0, 1 - np.sum((trues-predicted)**2) / np.sum((trues - np.mean(trues))**2))
    return corr, mae, mse, rae, rmse, r2

In [9]:
def eval_quantiles(lower, upper, trues, preds):
    N = len(trues)
    icp = 1.0*np.sum((trues>lower) & (trues<upper)) / N
    diffs = np.maximum(0, upper-lower)
    mil = np.sum(diffs) / N
    rmil = 0.0
    for i in xrange(N):
        if trues[i] != preds[i]:
            rmil += diffs[i] / (np.abs(trues[i]-preds[i]))
    rmil = rmil / N
    clc = np.exp(-rmil*(icp-0.95))
    return icp, mil, rmil, clc

## Linear regression baseline

In [10]:
# place id of interest
ix = 7
print ids[ix]

aM


In [11]:
# true values we want to predict
y_true = y_test[:,0,ix,0,0] * removed_std_all[ix][n_train:n_test] + removed_seasonality_all[ix][n_train:n_test]

# naive baseline
preds_naive = y[n_train-STEPS_AHEAD:n_test-STEPS_AHEAD,0,ix,0,0] * removed_std_all[ix][n_train:n_test] + removed_seasonality_all[ix][n_train:n_test]
corr, mae_naive, mse, rae, rmse_naive, r2_naive = compute_error(y_true, preds_naive)
print "NAIVE: %.3f   %.3f   %.3f" % (mae_naive,rmse_naive,r2_naive)

# linear regression
regr = linear_model.LinearRegression()
regr.fit(X_train[:,:,ix,0,0], y_train[:,0,ix,0,0])
preds_lr = regr.predict(X_test[:,:,ix,0,0]) * removed_std_all[ix][n_train:n_test] + removed_seasonality_all[ix][n_train:n_test]

corr, mae_lr, mse, rae, rmse_lr, r2_lr = compute_error(y_true, preds_lr)
print "LR:    %.3f   %.3f   %.3f" % (mae_lr,rmse_lr,r2_lr)

NAIVE: 2.808   4.920   0.724
LR:    2.287   4.245   0.794


## Specify model

In [12]:
def tilted_loss(q,y,f):
    e = (y-f)
    # The term inside k.mean is a one line simplification of the first equation
    return K.mean(q*e + K.clip(-e, K.epsilon(), np.inf), axis=-1)

In [13]:
def build_model(loss="mse", num_outs=1):
    model = Sequential()

    #model.add(LSTM(layers[1], input_shape=(None, layers[0]), return_sequences=True))
    model.add(ConvLSTM2D(filters=20, kernel_size=(3, 3),
                   input_shape=(None, 9, 1, 1),
                   padding='same', return_sequences=True))
    model.add(Dropout(0.2))

    model.add(ConvLSTM2D(filters=20, kernel_size=(3, 3),
                       padding='same', return_sequences=False))
    
    #model.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
    #           activation='linear',
    #           padding='same', data_format='channels_last'))

    #model.add(LSTM(layers[2], return_sequences=False))
    model.add(Dropout(0.2))
    
    #model.add(Dense(units=1, kernel_regularizer=regularizers.l2(0.00001)))
    model.add(Dense(units=num_outs))
    model.add(Activation("linear"))

    start = time.time()
    model.compile(loss=loss, optimizer="rmsprop")
    
    return model

## Train model and test

In [14]:
print X_train.shape
print X_train[:,:,:,0,0].transpose([0,2,1]).shape
print y_train.shape

(25632, 50, 9, 1, 1)
(25632, 9, 50)
(25632, 1, 9, 1, 1)


In [15]:
model = build_model(num_outs=1)

# checkpoint best model
checkpoint = ModelCheckpoint('ordered_places_convlstm_mean.best.hdf5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

model.fit(
    X_train,
    y_train[:,:,:,:,0].swapaxes(1,2),
    batch_size=512,
    epochs=100,
    validation_split=0.2,
    callbacks=[checkpoint],
    verbose=2)  

Train on 20505 samples, validate on 5127 samples
Epoch 1/100
Epoch 00000: val_loss improved from inf to 0.55484, saving model to ordered_places_convlstm_mean.best.hdf5
13s - loss: 0.9312 - val_loss: 0.5548
Epoch 2/100
Epoch 00001: val_loss improved from 0.55484 to 0.38327, saving model to ordered_places_convlstm_mean.best.hdf5
12s - loss: 0.5781 - val_loss: 0.3833
Epoch 3/100
Epoch 00002: val_loss improved from 0.38327 to 0.36019, saving model to ordered_places_convlstm_mean.best.hdf5
13s - loss: 0.4530 - val_loss: 0.3602
Epoch 4/100
Epoch 00003: val_loss improved from 0.36019 to 0.30259, saving model to ordered_places_convlstm_mean.best.hdf5
12s - loss: 0.4063 - val_loss: 0.3026
Epoch 5/100
Epoch 00004: val_loss improved from 0.30259 to 0.27276, saving model to ordered_places_convlstm_mean.best.hdf5
12s - loss: 0.3705 - val_loss: 0.2728
Epoch 6/100
Epoch 00005: val_loss improved from 0.27276 to 0.26968, saving model to ordered_places_convlstm_mean.best.hdf5
12s - loss: 0.3432 - val_lo

Epoch 00076: val_loss improved from 0.22205 to 0.22190, saving model to ordered_places_convlstm_mean.best.hdf5
13s - loss: 0.2734 - val_loss: 0.2219
Epoch 78/100
Epoch 00077: val_loss improved from 0.22190 to 0.22176, saving model to ordered_places_convlstm_mean.best.hdf5
12s - loss: 0.2733 - val_loss: 0.2218
Epoch 79/100
Epoch 00078: val_loss did not improve
12s - loss: 0.2726 - val_loss: 0.2283
Epoch 80/100
Epoch 00079: val_loss did not improve
12s - loss: 0.2727 - val_loss: 0.2292
Epoch 81/100
Epoch 00080: val_loss did not improve
12s - loss: 0.2731 - val_loss: 0.2323
Epoch 82/100
Epoch 00081: val_loss did not improve
12s - loss: 0.2732 - val_loss: 0.2243
Epoch 83/100
Epoch 00082: val_loss did not improve
12s - loss: 0.2718 - val_loss: 0.2233
Epoch 84/100
Epoch 00083: val_loss did not improve
13s - loss: 0.2729 - val_loss: 0.2263
Epoch 85/100
Epoch 00084: val_loss did not improve
13s - loss: 0.2725 - val_loss: 0.2237
Epoch 86/100
Epoch 00085: val_loss did not improve
12s - loss: 0.2

In [16]:
# load weights
model.load_weights('ordered_places_convlstm_mean.best.hdf5')

# make predictions
predictions = model.predict(X_test)
print predictions.shape

(25920, 9, 1, 1)


## Train independent NNs for the quantiles

In [17]:
quantiles = [0.05, 0.10, 0.90, 0.95]

In [18]:
models_q = []
for q in quantiles:
    print "training NN for quantile:", q
    
    # checkpoint best model
    checkpoint = ModelCheckpoint("ordered_places_convlstm_q%2f.best.hdf5" % (q,), monitor='val_loss', verbose=0, save_best_only=True, mode='min')

    # fit model to the 5% quantile
    model_q = build_model(loss=lambda y,f: tilted_loss(q,y,f))
    model_q.fit(
        X_train,
        y_train[:,:,:,:,0].swapaxes(1,2),
        batch_size=512,
        epochs=100,
        validation_split=0.2,
        callbacks=[checkpoint],
        verbose=2)  
    models_q.append(model_q)

training NN for quantile: 0.05
Train on 20505 samples, validate on 5127 samples
Epoch 1/100
14s - loss: 0.1502 - val_loss: 0.0946
Epoch 2/100
12s - loss: 0.1001 - val_loss: 0.0893
Epoch 3/100
12s - loss: 0.0870 - val_loss: 0.0836
Epoch 4/100
13s - loss: 0.0768 - val_loss: 0.0747
Epoch 5/100
13s - loss: 0.0686 - val_loss: 0.0709
Epoch 6/100
13s - loss: 0.0639 - val_loss: 0.0606
Epoch 7/100
12s - loss: 0.0609 - val_loss: 0.0541
Epoch 8/100
13s - loss: 0.0591 - val_loss: 0.0489
Epoch 9/100
12s - loss: 0.0567 - val_loss: 0.0480
Epoch 10/100
12s - loss: 0.0550 - val_loss: 0.0493
Epoch 11/100
12s - loss: 0.0538 - val_loss: 0.0441
Epoch 12/100
13s - loss: 0.0524 - val_loss: 0.0452
Epoch 13/100
13s - loss: 0.0515 - val_loss: 0.0487
Epoch 14/100
12s - loss: 0.0502 - val_loss: 0.0456
Epoch 15/100
12s - loss: 0.0498 - val_loss: 0.0409
Epoch 16/100
12s - loss: 0.0490 - val_loss: 0.0408
Epoch 17/100
13s - loss: 0.0485 - val_loss: 0.0396
Epoch 18/100
13s - loss: 0.0478 - val_loss: 0.0396
Epoch 19/10

13s - loss: 0.0658 - val_loss: 0.0591
Epoch 60/100
12s - loss: 0.0658 - val_loss: 0.0583
Epoch 61/100
13s - loss: 0.0656 - val_loss: 0.0569
Epoch 62/100
12s - loss: 0.0658 - val_loss: 0.0587
Epoch 63/100
12s - loss: 0.0656 - val_loss: 0.0576
Epoch 64/100
13s - loss: 0.0656 - val_loss: 0.0564
Epoch 65/100
13s - loss: 0.0654 - val_loss: 0.0566
Epoch 66/100
12s - loss: 0.0654 - val_loss: 0.0584
Epoch 67/100
12s - loss: 0.0654 - val_loss: 0.0575
Epoch 68/100
13s - loss: 0.0654 - val_loss: 0.0571
Epoch 69/100
12s - loss: 0.0655 - val_loss: 0.0560
Epoch 70/100
12s - loss: 0.0653 - val_loss: 0.0602
Epoch 71/100
13s - loss: 0.0653 - val_loss: 0.0587
Epoch 72/100
13s - loss: 0.0652 - val_loss: 0.0577
Epoch 73/100
13s - loss: 0.0652 - val_loss: 0.0585
Epoch 74/100
13s - loss: 0.0651 - val_loss: 0.0606
Epoch 75/100
13s - loss: 0.0652 - val_loss: 0.0577
Epoch 76/100
12s - loss: 0.0652 - val_loss: 0.0567
Epoch 77/100
13s - loss: 0.0650 - val_loss: 0.0563
Epoch 78/100
13s - loss: 0.0650 - val_loss: 

13s - loss: 0.0486 - val_loss: 0.0437
Epoch 18/100
13s - loss: 0.0481 - val_loss: 0.0427
Epoch 19/100
13s - loss: 0.0478 - val_loss: 0.0471
Epoch 20/100
12s - loss: 0.0474 - val_loss: 0.0450
Epoch 21/100
13s - loss: 0.0469 - val_loss: 0.0422
Epoch 22/100
12s - loss: 0.0466 - val_loss: 0.0407
Epoch 23/100
13s - loss: 0.0465 - val_loss: 0.0405
Epoch 24/100
12s - loss: 0.0460 - val_loss: 0.0413
Epoch 25/100
13s - loss: 0.0460 - val_loss: 0.0523
Epoch 26/100
13s - loss: 0.0462 - val_loss: 0.0399
Epoch 27/100
12s - loss: 0.0459 - val_loss: 0.0408
Epoch 28/100
13s - loss: 0.0454 - val_loss: 0.0415
Epoch 29/100
13s - loss: 0.0452 - val_loss: 0.0405
Epoch 30/100
12s - loss: 0.0451 - val_loss: 0.0422
Epoch 31/100
13s - loss: 0.0450 - val_loss: 0.0400
Epoch 32/100
12s - loss: 0.0449 - val_loss: 0.0413
Epoch 33/100
12s - loss: 0.0447 - val_loss: 0.0399
Epoch 34/100
13s - loss: 0.0446 - val_loss: 0.0395
Epoch 35/100
12s - loss: 0.0445 - val_loss: 0.0403
Epoch 36/100
13s - loss: 0.0444 - val_loss: 

In [19]:
predictions_q = []
for i in xrange(len(quantiles)):
    q = quantiles[i]
    print "making predictions for quantile:", q
    
    # load weights
    models_q[i].load_weights("ordered_places_convlstm_q%2f.best.hdf5" % (q,))

    # make predictions
    predictions_q.append(models_q[i].predict(X_test))

making predictions for quantile: 0.05
making predictions for quantile: 0.1
making predictions for quantile: 0.9
making predictions for quantile: 0.95


## Linear Quantile Regression baselines

In [20]:
print X_train.shape
print y_train.shape

(25632, 50, 9, 1, 1)
(25632, 1, 9, 1, 1)


In [21]:
import warnings
warnings.filterwarnings("ignore")

col_names = ["l%d" % (i,) for i in xrange(NUM_LAGS-STEPS_AHEAD+1)]
col_names.append("y")
qr_str = "y ~ " + ''.join(["+l%d" % (x,) for x in range(NUM_LAGS-STEPS_AHEAD+1)])[1:]

predictions_lr = []
predictions_q_lr = []
for k in xrange(len(quantiles)):
    q = quantiles[k]
    print "running quantile regression for quantile:", q
    
    preds_lr = np.zeros((n_test-n_train, n_places))
    preds_qr = np.zeros((n_test-n_train, n_places))
    for i in xrange(n_places):
        regr = linear_model.LinearRegression()
        regr.fit(X_train[:,:,i,0,0], y_train[:,0,ix,0,0])
        preds_lr[:,i] = regr.predict(X_test[:,:,i,0,0]) * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]

        data = pd.DataFrame(data = np.hstack([X_train[:,:,i,0,0], y_train[:,0,ix,0,:]]), columns = col_names)
        data_test = pd.DataFrame(data = np.hstack([X_test[:,:,i,0,0], y_test[:,0,ix,0,:]]), columns = col_names)

        #qr = smf.quantreg('y ~ l0+l1+l2+l3+l4+l5+l6+l7+l8', data)
        qr = smf.quantreg(qr_str, data)
        res = qr.fit(q=q)
        preds_qr[:,i] = res.predict(data_test) * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    
    predictions_lr.append(preds_lr)
    predictions_q_lr.append(preds_qr)

running quantile regression for quantile: 0.05
running quantile regression for quantile: 0.1
running quantile regression for quantile: 0.9
running quantile regression for quantile: 0.95


## Simultaneous Quantiles NN

In [22]:
def multi_tilted_loss(quantiles,y,f):
    print y
    print f
    # The term inside k.mean is a one line simplification of the first equation
    loss = K.mean(K.square(y[:,:,:,0]-f[:,:,:,0]), axis=-1)
    for k in xrange(len(quantiles)):
        q = quantiles[k]
        e = (y[:,:,:,k+1]-f[:,:,:,k+1])
        loss += K.mean(q*e + K.clip(-e, K.epsilon(), np.inf), axis=-1)
    return loss

In [23]:
global previous_losses
previous_losses = [1.0,1.0,1.0,1.0,1.0]
def multi_tilted_loss_weighted(quantiles,y,f):
    global previous_losses
    print y
    print f
    max_prev_loss = tf.reduce_max(previous_losses)
    
    # The term inside k.mean is a one line simplification of the first equation
    losses = [(max_prev_loss/previous_losses[0]) * K.mean(K.square(y[:,:,:,0]-f[:,:,:,0]), axis=-1)]
    for k in xrange(len(quantiles)):
        q = quantiles[k]
        e = (y[:,:,:,k+1]-f[:,:,:,k+1])
        losses.append((max_prev_loss/previous_losses[k+1]) * K.mean(q*e + K.clip(-e, K.epsilon(), np.inf), axis=-1))
    previous_losses = losses
    loss = tf.reduce_sum(losses)
    return loss

In [24]:
print y_train[:,:,:,:,0].swapaxes(1,2).shape

(25632, 9, 1, 1)


In [25]:
y_traink = y_train[:,:,:,:,0].swapaxes(1,2)
for k in xrange(len(quantiles)):
    y_traink = np.concatenate((y_traink, y_train[:,:,:,:,0].swapaxes(1,2)), axis=3)
print y_traink.shape

(25632, 9, 1, 5)


In [26]:
# checkpoint best model
checkpoint = ModelCheckpoint("ordered_places_convlstm_multi.best.hdf5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# fit model to the 95% quantile
model_multi = build_model(loss=lambda y,f: multi_tilted_loss_weighted(quantiles,y,f), num_outs=1+len(quantiles))
model_multi.fit(
    X_train,
    y_traink,
    batch_size=128,
    epochs=80,
    validation_split=0.2,
    callbacks=[checkpoint],
    verbose=2)

Tensor("activation_6_target:0", shape=(?, ?, ?, ?), dtype=float32)
Tensor("activation_6/Identity:0", shape=(?, 9, 1, 5), dtype=float32)
Train on 20505 samples, validate on 5127 samples
Epoch 1/80
Epoch 00000: val_loss improved from inf to 732.46574, saving model to ordered_places_convlstm_multi.best.hdf5
78s - loss: 1449.0801 - val_loss: 732.4657
Epoch 2/80
Epoch 00001: val_loss improved from 732.46574 to 584.77703, saving model to ordered_places_convlstm_multi.best.hdf5
53s - loss: 835.2225 - val_loss: 584.7770
Epoch 3/80
Epoch 00002: val_loss improved from 584.77703 to 539.70156, saving model to ordered_places_convlstm_multi.best.hdf5
53s - loss: 703.9059 - val_loss: 539.7016
Epoch 4/80
Epoch 00003: val_loss improved from 539.70156 to 518.33632, saving model to ordered_places_convlstm_multi.best.hdf5
53s - loss: 666.8412 - val_loss: 518.3363
Epoch 5/80
Epoch 00004: val_loss improved from 518.33632 to 506.82580, saving model to ordered_places_convlstm_multi.best.hdf5
53s - loss: 650.8

Epoch 00067: val_loss did not improve
54s - loss: 563.4348 - val_loss: 475.2345
Epoch 69/80
Epoch 00068: val_loss did not improve
54s - loss: 564.7405 - val_loss: 472.7862
Epoch 70/80
Epoch 00069: val_loss did not improve
53s - loss: 564.6941 - val_loss: 472.4743
Epoch 71/80
Epoch 00070: val_loss did not improve
53s - loss: 562.9162 - val_loss: 479.6676
Epoch 72/80
Epoch 00071: val_loss did not improve
54s - loss: 562.6623 - val_loss: 476.2814
Epoch 73/80
Epoch 00072: val_loss did not improve
54s - loss: 564.9753 - val_loss: 476.1068
Epoch 74/80
Epoch 00073: val_loss did not improve
55s - loss: 562.9764 - val_loss: 473.2004
Epoch 75/80
Epoch 00074: val_loss did not improve
54s - loss: 563.0736 - val_loss: 473.5396
Epoch 76/80
Epoch 00075: val_loss did not improve
54s - loss: 562.8013 - val_loss: 473.8369
Epoch 77/80
Epoch 00076: val_loss did not improve
54s - loss: 561.2921 - val_loss: 478.9492
Epoch 78/80
Epoch 00077: val_loss did not improve
54s - loss: 561.5855 - val_loss: 474.0167


In [27]:
a = 1

In [28]:
# load weights
model_multi.load_weights("ordered_places_convlstm_multi.best.hdf5")

# make predictions
predictions_multi = model_multi.predict(X_test)
print predictions_multi.shape

(25920, 9, 1, 5)


## Compare results for different methods

In [29]:
predictions_multi.shape

(25920, 9, 1, 5)

In [30]:
def compare_methods(i):
    trues = y_test[:,0,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    preds_qr = predictions_lr[0][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    lower_qr1 = predictions_q_lr[0][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    lower_qr2 = predictions_q_lr[1][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    upper_qr1 = predictions_q_lr[2][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    upper_qr2 = predictions_q_lr[3][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]

    preds_dl = predictions[:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_dl1 = predictions_q[0][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_dl2 = predictions_q[1][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_dl1 = predictions_q[2][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_dl2 = predictions_q[3][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]

    preds_multi = predictions_multi[:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_multi1 = predictions_multi[:,i,0,1] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_multi2 = predictions_multi[:,i,0,2] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_multi1 = predictions_multi[:,i,0,3] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_multi2 = predictions_multi[:,i,0,4] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]

    # evaluate prediction means
    print "Prediction mean:"
    print "\tMAE\tRMSE\tR2"
    corr, mae, mse, rae, rmse, r2 = compute_error(trues, preds_qr)
    print "LR: \t%.3f\t%.3f\t%.3f" % (mae,rmse,r2)
    corr, mae, mse, rae, rmse, r2 = compute_error(trues, preds_dl)
    print "DLQR: \t%.3f\t%.3f\t%.3f" % (mae,rmse,r2)
    corr, mae, mse, rae, rmse, r2 = compute_error(trues, preds_multi)
    print "MULTI: \t%.3f\t%.3f\t%.3f" % (mae,rmse,r2)

    # evaluate quantiles
    print "\nQuantiles 90%:"
    print "\tICP\tMIL\tRMIL"
    icp, mil, rmil, clc = eval_quantiles(lower_qr1, upper_qr2, trues, preds_qr)
    print "QR: \t%.3f\t%.3f\t%.3f" % (icp, mil, rmil)
    icp, mil, rmil, clc = eval_quantiles(lower_dl1, upper_dl2, trues, preds_dl)
    print "DLQR: \t%.3f\t%.3f\t%.3f" % (icp, mil, rmil)
    icp, mil, rmil, clc = eval_quantiles(lower_multi1, upper_multi2, trues, preds_multi)
    print "MULTI: \t%.3f\t%.3f\t%.3f" % (icp, mil, rmil)

    print "\nQuantiles 60%:"
    print "\tICP\tMIL\tRMIL"
    icp, mil, rmil, clc = eval_quantiles(lower_qr2, upper_qr1, trues, preds_qr)
    print "QR: \t%.3f\t%.3f\t%.3f" % (icp, mil, rmil)
    icp, mil, rmil, clc = eval_quantiles(lower_dl2, upper_dl1, trues, preds_dl)
    print "DLQR: \t%.3f\t%.3f\t%.3f" % (icp, mil, rmil)
    icp, mil, rmil, clc = eval_quantiles(lower_multi2, upper_multi1, trues, preds_multi)
    print "MULTI: \t%.3f\t%.3f\t%.3f" % (icp, mil, rmil)

In [31]:
compare_methods(ix)

Prediction mean:
	MAE	RMSE	R2
LR: 	2.287	4.245	0.794
DLQR: 	1.952	3.855	0.830
MULTI: 	1.940	3.832	0.832

Quantiles 90%:
	ICP	MIL	RMIL
QR: 	0.909	10.687	73.365
DLQR: 	0.897	8.015	53.522
MULTI: 	0.933	8.574	32.638

Quantiles 60%:
	ICP	MIL	RMIL
QR: 	0.819	6.337	43.235
DLQR: 	0.788	5.793	36.340
MULTI: 	0.852	6.253	24.161


In [32]:
maes_lr = []
rmses_lr = []
r2s_lr = []
maes_dl = []
rmses_dl = []
r2s_dl = []
maes_multi = []
rmses_multi = []
r2s_multi = []
for i in xrange(n_places):
    trues = y_test[:,0,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    preds_qr = predictions_lr[0][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    lower_qr1 = predictions_q_lr[0][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    lower_qr2 = predictions_q_lr[1][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    upper_qr1 = predictions_q_lr[2][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    upper_qr2 = predictions_q_lr[3][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]

    preds_dl = predictions[:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_dl1 = predictions_q[0][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_dl2 = predictions_q[1][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_dl1 = predictions_q[2][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_dl2 = predictions_q[3][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]

    preds_multi = predictions_multi[:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_multi1 = predictions_multi[:,i,0,1] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_multi2 = predictions_multi[:,i,0,2] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_multi1 = predictions_multi[:,i,0,3] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_multi2 = predictions_multi[:,i,0,4] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]

    # evaluate prediction means
    s = "[%d]\tMean:" % (i,)
    corr, mae, mse, rae, rmse, r2 = compute_error(trues, preds_qr)
    maes_lr.append(mae)
    rmses_lr.append(rmse)
    r2s_lr.append(r2)
    s += "\t(QR: %.2f %.2f %.2f)" % (mae,rmse,r2)
    corr, mae, mse, rae, rmse, r2 = compute_error(trues, preds_dl)
    maes_dl.append(mae)
    rmses_dl.append(rmse)
    r2s_dl.append(r2)
    s += "\t(DLQR: %.2f %.2f %.2f)  " % (mae,rmse,r2)
    corr, mae, mse, rae, rmse, r2 = compute_error(trues, preds_multi)
    maes_multi.append(mae)
    rmses_multi.append(rmse)
    r2s_multi.append(r2)
    s += "\t(MULTI: %.2f %.2f %.2f)" % (mae,rmse,r2)
    print s

[0]	Mean:	(QR: 3.43 5.29 0.54)	(DLQR: 1.74 3.49 0.80)  	(MULTI: 1.73 3.46 0.80)
[1]	Mean:	(QR: 3.79 5.74 0.57)	(DLQR: 1.92 3.71 0.82)  	(MULTI: 1.90 3.69 0.82)
[2]	Mean:	(QR: 3.64 5.70 0.57)	(DLQR: 1.88 3.70 0.82)  	(MULTI: 1.87 3.68 0.82)
[3]	Mean:	(QR: 3.61 5.64 0.61)	(DLQR: 1.95 3.93 0.81)  	(MULTI: 1.94 3.91 0.81)
[4]	Mean:	(QR: 3.25 5.11 0.70)	(DLQR: 1.94 3.81 0.84)  	(MULTI: 1.94 3.78 0.84)
[5]	Mean:	(QR: 3.01 4.84 0.73)	(DLQR: 1.97 3.86 0.83)  	(MULTI: 1.97 3.84 0.83)
[6]	Mean:	(QR: 2.69 4.49 0.77)	(DLQR: 1.98 3.89 0.83)  	(MULTI: 1.96 3.85 0.83)
[7]	Mean:	(QR: 2.29 4.24 0.79)	(DLQR: 1.95 3.85 0.83)  	(MULTI: 1.94 3.83 0.83)
[8]	Mean:	(QR: 2.71 4.44 0.79)	(DLQR: 1.88 3.69 0.85)  	(MULTI: 1.88 3.68 0.85)


In [33]:
print "ERRORS:"
print "LR:    %.3f %.3f %.3f" % (np.mean(maes_lr),np.mean(rmses_lr),np.mean(r2s_lr)) 
print "DL:    %.3f %.3f %.3f" % (np.mean(maes_dl),np.mean(rmses_dl),np.mean(r2s_dl)) 
print "MULTI: %.3f %.3f %.3f" % (np.mean(maes_multi),np.mean(rmses_multi),np.mean(r2s_multi)) 

ERRORS:
LR:    3.157 5.055 0.674
DL:    1.912 3.768 0.825
MULTI: 1.904 3.747 0.827


In [34]:
icps_qr = []
mils_qr = []
rmils_qr = []
icps_dl = []
mils_dl = []
rmils_dl = []
icps_multi = []
mils_multi = []
rmils_multi = []
for i in xrange(n_places):
    trues = y_test[:,0,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    preds_qr = predictions_lr[0][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    lower_qr1 = predictions_q_lr[0][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    lower_qr2 = predictions_q_lr[1][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    upper_qr1 = predictions_q_lr[2][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    upper_qr2 = predictions_q_lr[3][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]

    preds_dl = predictions[:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_dl1 = predictions_q[0][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_dl2 = predictions_q[1][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_dl1 = predictions_q[2][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_dl2 = predictions_q[3][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]

    preds_multi = predictions_multi[:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_multi1 = predictions_multi[:,i,0,1] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_multi2 = predictions_multi[:,i,0,2] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_multi1 = predictions_multi[:,i,0,3] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_multi2 = predictions_multi[:,i,0,4] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]


    # evaluate quantiles
    s = "[%d]\tQuantiles 95:" % (i,)
    icp, mil, rmil, clc = eval_quantiles(lower_qr1, upper_qr2, trues, preds_qr)
    icps_qr.append(icp)
    mils_qr.append(mil)
    rmils_qr.append(rmil)
    s += "\t(QR: %.2f %.2f %.2f)" % (icp, mil, rmil)
    icp, mil, rmil, clc = eval_quantiles(lower_dl1, upper_dl2, trues, preds_dl)
    icps_dl.append(icp)
    mils_dl.append(mil)
    rmils_dl.append(rmil)
    s += "\t(DLQR: %.2f %.2f %.2f)  " % (icp, mil, rmil)
    icp, mil, rmil, clc = eval_quantiles(lower_multi1, upper_multi2, trues, preds_multi)
    icps_multi.append(icp)
    mils_multi.append(mil)
    rmils_multi.append(rmil)
    s += "\t(MULTI: %.2f %.2f %.2f)" % (icp, mil, rmil)
    print s

[0]	Quantiles 95:	(QR: 0.94 21.68 72.02)	(DLQR: 0.91 7.42 38.56)  	(MULTI: 0.93 7.68 98.47)
[1]	Quantiles 95:	(QR: 0.94 23.48 89.76)	(DLQR: 0.89 8.02 31.69)  	(MULTI: 0.94 8.66 38.45)
[2]	Quantiles 95:	(QR: 0.94 23.30 54.62)	(DLQR: 0.90 7.95 45.18)  	(MULTI: 0.93 8.38 57.53)
[3]	Quantiles 95:	(QR: 0.95 23.13 58.75)	(DLQR: 0.89 8.27 38.27)  	(MULTI: 0.93 8.69 40.34)
[4]	Quantiles 95:	(QR: 0.95 21.83 1298.88)	(DLQR: 0.89 8.30 27.27)  	(MULTI: 0.93 8.79 28.53)
[5]	Quantiles 95:	(QR: 0.95 19.71 72.73)	(DLQR: 0.90 8.33 39.98)  	(MULTI: 0.93 8.68 71.87)
[6]	Quantiles 95:	(QR: 0.94 15.76 82.51)	(DLQR: 0.90 8.23 29.21)  	(MULTI: 0.93 8.58 54.57)
[7]	Quantiles 95:	(QR: 0.91 10.69 73.37)	(DLQR: 0.90 8.02 53.52)  	(MULTI: 0.93 8.57 32.64)
[8]	Quantiles 95:	(QR: 0.95 16.77 87.62)	(DLQR: 0.91 8.16 45.89)  	(MULTI: 0.94 8.52 755.09)


In [35]:
print "QUANTILES 95 (MEAN):"
print "QR:    %.3f %.3f %.3f" % (np.mean(icps_qr),np.mean(mils_qr),np.mean(rmils_qr)) 
print "DLQR:  %.3f %.3f %.3f" % (np.mean(icps_dl),np.mean(mils_dl),np.mean(rmils_dl)) 
print "MULTI: %.3f %.3f %.3f" % (np.mean(icps_multi),np.mean(mils_multi),np.mean(rmils_multi)) 

QUANTILES 95 (MEAN):
QR:    0.941 19.594 210.029
DLQR:  0.900 8.077 38.840
MULTI: 0.933 8.506 130.832


In [36]:
print "QUANTILES 90 (MEDIAN):"
print "QR:    %.3f %.3f %.3f" % (np.median(icps_qr),np.median(mils_qr),np.median(rmils_qr)) 
print "DLQR:  %.3f %.3f %.3f" % (np.median(icps_dl),np.median(mils_dl),np.median(rmils_dl)) 
print "MULTI: %.3f %.3f %.3f" % (np.median(icps_multi),np.median(mils_multi),np.median(rmils_multi)) 

QUANTILES 90 (MEDIAN):
QR:    0.945 21.677 73.365
DLQR:  0.899 8.155 38.555
MULTI: 0.934 8.577 54.567


In [37]:
icps_qr = []
mils_qr = []
rmils_qr = []
icps_dl = []
mils_dl = []
rmils_dl = []
icps_multi = []
mils_multi = []
rmils_multi = []
for i in xrange(n_places):
    trues = y_test[:,0,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    preds_qr = predictions_lr[0][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    lower_qr1 = predictions_q_lr[0][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    lower_qr2 = predictions_q_lr[1][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    upper_qr1 = predictions_q_lr[2][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    upper_qr2 = predictions_q_lr[3][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]

    preds_dl = predictions[:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_dl1 = predictions_q[0][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_dl2 = predictions_q[1][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_dl1 = predictions_q[2][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_dl2 = predictions_q[3][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]

    preds_multi = predictions_multi[:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_multi1 = predictions_multi[:,i,0,1] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_multi2 = predictions_multi[:,i,0,2] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_multi1 = predictions_multi[:,i,0,3] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_multi2 = predictions_multi[:,i,0,4] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]


    # evaluate quantiles
    s = "[%d]\tQuantiles 60:" % (i,)
    icp, mil, rmil, clc = eval_quantiles(lower_qr2, upper_qr1, trues, preds_qr)
    icps_qr.append(icp)
    mils_qr.append(mil)
    rmils_qr.append(rmil)
    s += "\t(QR: %.2f %.2f %.2f)" % (icp, mil, rmil)
    icp, mil, rmil, clc = eval_quantiles(lower_dl2, upper_dl1, trues, preds_dl)
    icps_dl.append(icp)
    mils_dl.append(mil)
    rmils_dl.append(rmil)
    s += "\t(DLQR: %.2f %.2f %.2f)  " % (icp, mil, rmil)
    icp, mil, rmil, clc = eval_quantiles(lower_multi2, upper_multi1, trues, preds_multi)
    icps_multi.append(icp)
    mils_multi.append(mil)
    rmils_multi.append(rmil)
    s += "\t(MULTI: %.2f %.2f %.2f)" % (icp, mil, rmil)
    print s

[0]	Quantiles 60:	(QR: 0.88 13.54 45.35)	(DLQR: 0.81 5.37 28.74)  	(MULTI: 0.85 5.58 73.33)
[1]	Quantiles 60:	(QR: 0.87 14.38 54.83)	(DLQR: 0.79 5.86 23.50)  	(MULTI: 0.85 6.31 28.39)
[2]	Quantiles 60:	(QR: 0.89 14.26 33.61)	(DLQR: 0.79 5.66 32.31)  	(MULTI: 0.85 6.11 42.57)
[3]	Quantiles 60:	(QR: 0.88 14.18 36.10)	(DLQR: 0.79 5.92 27.55)  	(MULTI: 0.85 6.33 29.52)
[4]	Quantiles 60:	(QR: 0.89 12.95 774.81)	(DLQR: 0.79 6.04 20.31)  	(MULTI: 0.85 6.40 21.07)
[5]	Quantiles 60:	(QR: 0.89 11.29 40.99)	(DLQR: 0.79 5.97 29.81)  	(MULTI: 0.85 6.33 53.68)
[6]	Quantiles 60:	(QR: 0.87 9.25 49.25)	(DLQR: 0.78 5.85 21.24)  	(MULTI: 0.84 6.25 40.23)
[7]	Quantiles 60:	(QR: 0.82 6.34 43.24)	(DLQR: 0.79 5.79 36.34)  	(MULTI: 0.85 6.25 24.16)
[8]	Quantiles 60:	(QR: 0.88 10.07 53.20)	(DLQR: 0.81 5.84 31.16)  	(MULTI: 0.86 6.20 565.92)


In [38]:
print "QUANTILES 60:"
print "QR:    %.3f %.3f %.3f" % (np.mean(icps_qr),np.mean(mils_qr),np.mean(rmils_qr)) 
print "DLQR:  %.3f %.3f %.3f" % (np.mean(icps_dl),np.mean(mils_dl),np.mean(rmils_dl)) 
print "MULTI: %.3f %.3f %.3f" % (np.mean(icps_multi),np.mean(mils_multi),np.mean(rmils_multi)) 

QUANTILES 60:
QR:    0.874 11.807 125.708
DLQR:  0.793 5.813 27.883
MULTI: 0.850 6.194 97.654


In [39]:
print "QUANTILES 60:"
print "QR:    %.3f %.3f %.3f" % (np.median(icps_qr),np.median(mils_qr),np.median(rmils_qr)) 
print "DLQR:  %.3f %.3f %.3f" % (np.median(icps_dl),np.median(mils_dl),np.median(rmils_dl)) 
print "MULTI: %.3f %.3f %.3f" % (np.median(icps_multi),np.median(mils_multi),np.median(rmils_multi)) 

QUANTILES 60:
QR:    0.884 12.948 45.347
DLQR:  0.788 5.853 28.737
MULTI: 0.850 6.253 40.235


In [40]:
loss_qr = 0.0
loss_dl = 0.0
loss_multi = 0.0
for i in xrange(n_places):
     for k in xrange(len(quantiles)-1):
        q1 = predictions_q_lr[k][:,i]
        q2 = predictions_q_lr[k+1][:,i]
        loss_qr += np.sum(np.maximum(0.0, q1 - q2)) / len(q1)

        q1 = predictions_q[k][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
        q2 = predictions_q[k+1][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
        loss_dl += np.sum(np.maximum(0.0, q1 - q2)) / len(q1)

        q1 = predictions_multi[:,i,0,1+k] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
        q2 = predictions_multi[:,i,0,1+k+1] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
        loss_multi += np.sum(np.maximum(0.0, q1 - q2)) / len(q1)

print "cross loss qr:", loss_qr
print "cross loss dl:", loss_dl
print "cross loss multi:", loss_multi

cross loss qr: 0.029357388637
cross loss dl: 0.441799907221
cross loss multi: 0.0


In [41]:
for k in xrange(len(quantiles)-1):
    loss_qr = np.sum(predictions_q_lr[k][:,ix] > predictions_q_lr[k+1][:,ix])
    loss_dl = np.sum(predictions_q[k][:,ix,0,0] > predictions_q[k+1][:,ix,0,0])
    loss_multi = np.sum(predictions_multi[:,ix,0,1+k] > predictions_multi[:,ix,0,1+k+1])
    print "[%d-%d] crossing cases: QR: %d\tDL: %d\t  MULTI: %d" % (k,k+1,loss_qr,loss_dl,loss_multi)

[0-1] crossing cases: QR: 53	DL: 1121	  MULTI: 0
[1-2] crossing cases: QR: 0	DL: 0	  MULTI: 0
[2-3] crossing cases: QR: 235	DL: 2127	  MULTI: 0


In [42]:
sum_qr = 0
sum_dl = 0
sum_multi = 0
for i in xrange(n_places):
    print "Road segment [%d]" % (i,)
    for k in xrange(len(quantiles)-1):
        loss_qr = np.sum(predictions_q_lr[k][:,i] > predictions_q_lr[k+1][:,i])
        sum_qr += loss_qr
        loss_dl = np.sum(predictions_q[k][:,i,0,0] > predictions_q[k+1][:,i,0,0])
        sum_dl += loss_dl
        loss_multi = np.sum(predictions_multi[:,i,0,1+k] > predictions_multi[:,i,0,1+k+1])
        sum_multi += loss_multi
        print "[%d-%d] crossing cases: QR: %d\tDL: %d\t  MULTI: %d" % (k,k+1,loss_qr,loss_dl,loss_multi)
print "sum_qr:", sum_qr
print "sum_dl:", sum_dl
print "sum_multi:", sum_multi

Road segment [0]
[0-1] crossing cases: QR: 2	DL: 1070	  MULTI: 0
[1-2] crossing cases: QR: 0	DL: 0	  MULTI: 0
[2-3] crossing cases: QR: 28	DL: 604	  MULTI: 0
Road segment [1]
[0-1] crossing cases: QR: 8	DL: 1203	  MULTI: 0
[1-2] crossing cases: QR: 4	DL: 0	  MULTI: 0
[2-3] crossing cases: QR: 45	DL: 1767	  MULTI: 0
Road segment [2]
[0-1] crossing cases: QR: 7	DL: 1347	  MULTI: 0
[1-2] crossing cases: QR: 0	DL: 0	  MULTI: 0
[2-3] crossing cases: QR: 15	DL: 1167	  MULTI: 0
Road segment [3]
[0-1] crossing cases: QR: 12	DL: 1432	  MULTI: 0
[1-2] crossing cases: QR: 1	DL: 0	  MULTI: 0
[2-3] crossing cases: QR: 23	DL: 1838	  MULTI: 0
Road segment [4]
[0-1] crossing cases: QR: 22	DL: 1753	  MULTI: 0
[1-2] crossing cases: QR: 0	DL: 0	  MULTI: 0
[2-3] crossing cases: QR: 44	DL: 2103	  MULTI: 0
Road segment [5]
[0-1] crossing cases: QR: 6	DL: 1416	  MULTI: 0
[1-2] crossing cases: QR: 0	DL: 0	  MULTI: 0
[2-3] crossing cases: QR: 79	DL: 1614	  MULTI: 0
Road segment [6]
[0-1] crossing cases: QR: 52